# Setup

In [1]:
import collections
import warnings
from time import perf_counter_ns

import numpy as np
import pandas as pd
import xgboost as xgb
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor

from asboostreg import SparseAdditiveBoostingRegressor

In [2]:
warnings.filterwarnings("ignore")

# Loading the Ames housing dataset

In [3]:
df_housing = pd.read_csv("http://dlsun.github.io/pods/data/AmesHousing.txt", sep="\t")
df_housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
X = df_housing.drop(columns=["SalePrice", "PID", "Order"])
y = df_housing["SalePrice"].astype(float)
y = pd.Series((y - y.mean()) / y.std(), index=y.index)
categorical_indices = np.array(
    [i for i, col in enumerate(X.columns) if X[col].dtype == "object"]
)
X_numeric = X.drop(columns=X.columns[categorical_indices])

In [5]:
X.head()
kf = KFold(n_splits=10, shuffle=True, random_state=0)
cv = list(kf.split(X))

# Defining the models

In [85]:
dummy = DummyRegressor()

# Interpretable but not strong
ridgereg = make_pipeline(
    OneHotEncoder(handle_unknown="ignore", sparse=False, max_categories=10),
    SimpleImputer(add_indicator=True),
    StandardScaler(),
    RidgeCV(cv=cv),
)  # Non Sparse
treereg = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    SimpleImputer(add_indicator=True),
    DecisionTreeRegressor(max_depth=4),
)  # Sparse

# Strong but not interpretable
rfreg = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    SimpleImputer(add_indicator=True),
    RandomForestRegressor(random_state=0),
)  # Non Sparse
xgbreg = xgb.XGBRegressor(random_state=0)  # Sparse

# Both interpretable and strong
ebmreg = ExplainableBoostingRegressor(interactions=0)  # Non Sparse
sparsereg = SparseAdditiveBoostingRegressor(
    learning_rate=0.01,
    n_estimators=10_000,
    l2_regularization=2.0,
    row_subsample=0.85,
    random_state=0,
    n_iter_no_change=15,
    max_bins=256,
    min_l0_fused_regularization=0.5,
    max_l0_fused_regularization=3.0,
    min_samples_leaf=2,
    validation_fraction=0.15,
)  # Sparse

# Comparing the models

In [86]:
# Running fast Hyperparameter optimization for Ridge
ridgereg.fit(X_numeric, y)
alpha = ridgereg.named_steps["ridgecv"].alpha_
ridgereg = make_pipeline(
    OneHotEncoder(handle_unknown="ignore", sparse=False, max_categories=10),
    SimpleImputer(add_indicator=True),
    StandardScaler(),
    Ridge(alpha=alpha),
)

In [87]:
def evaluate(model, X_train, X_test, y_train, y_test, **kwargs):
    start = perf_counter_ns()
    model.fit(X_train, y_train, **kwargs)
    end = perf_counter_ns()
    elapsed = (end - start) / 1e9
    train_error = mean_absolute_error(y_train, model.predict(X_train))
    dummy_train_error = mean_absolute_error(
        y_train, np.full(y_train.shape, y_train.median())
    )
    train_score = 1 - train_error / dummy_train_error
    test_error = mean_absolute_error(y_test, model.predict(X_test))
    dummy_test_error = mean_absolute_error(
        y_test, np.full(y_test.shape, y_train.median())
    )
    test_score = 1 - test_error / dummy_test_error
    name = (
        model.__class__.__name__
        if not isinstance(model, Pipeline)
        else model.steps[-1][1].__class__.__name__
    )
    print(
        f"{name}: {train_score:.3f} (train), {test_score:.3f} (test), {elapsed:.3f} (s)"
    )
    return test_score

In [88]:
test_scores = collections.defaultdict(list)

for i, (train_index, test_index) in enumerate(cv, 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(f"Fold {i}")
    print("------")

    test = evaluate(dummy, X_train, X_test, y_train, y_test)
    test_scores["Dummy"].append(test)

    test = evaluate(treereg, X_train, X_test, y_train, y_test)
    test_scores["Decision Tree"].append(test)

    test = evaluate(ridgereg, X_train, X_test, y_train, y_test)
    test_scores["Elastic Net"].append(test)

    X_train_numeric, X_test_numeric = (
        X_numeric.iloc[train_index],
        X_numeric.iloc[test_index],
    )

    test = evaluate(
        xgbreg,
        X_train_numeric,
        X_test_numeric,
        y_train,
        y_test,
        eval_set=[(X_test_numeric, y_test)],
        early_stopping_rounds=30,
        verbose=False,
    )
    test_scores["XGBoost"].append(test)

    test = evaluate(rfreg, X_train, X_test, y_train, y_test)
    test_scores["Random Forest"].append(test)

    test = evaluate(ebmreg, X_train, X_test, y_train, y_test)
    test_scores["EBM"].append(test)

    test = evaluate(
        sparsereg,
        X_train_numeric,
        X_test_numeric,
        y_train,
        y_test,
        validation_set=(X_test_numeric, y_test),
    )
    test_scores["SparseReg"].append(test)

    print()

Fold 1
------
DummyRegressor: -0.040 (train), -0.015 (test), 0.001 (s)
DecisionTreeRegressor: 0.556 (train), 0.387 (test), 0.132 (s)
Ridge: 0.715 (train), 0.633 (test), 0.338 (s)
XGBRegressor: 0.848 (train), 0.712 (test), 0.255 (s)
RandomForestRegressor: 0.894 (train), 0.501 (test), 20.740 (s)
ExplainableBoostingRegressor: 0.776 (train), 0.739 (test), 2.168 (s)
SparseAdditiveBoostingRegressor: 0.598 (train), 0.586 (test), 39.231 (s)

Fold 2
------
DummyRegressor: -0.039 (train), -0.045 (test), 0.000 (s)
DecisionTreeRegressor: 0.557 (train), 0.423 (test), 0.129 (s)
Ridge: 0.714 (train), 0.624 (test), 0.351 (s)
XGBRegressor: 0.865 (train), 0.728 (test), 0.352 (s)
RandomForestRegressor: 0.896 (train), 0.508 (test), 20.818 (s)
ExplainableBoostingRegressor: 0.747 (train), 0.693 (test), 1.308 (s)
SparseAdditiveBoostingRegressor: 0.596 (train), 0.560 (test), 17.768 (s)

Fold 3
------
DummyRegressor: -0.039 (train), -0.049 (test), 0.001 (s)
DecisionTreeRegressor: 0.558 (train), 0.453 (test), 0

In [ ]:
test_scores_df = pd.DataFrame(test_scores)
test_scores_df

In [ ]:
print(test_scores_df.drop("Dummy", axis=1).round(3).to_latex())